In [ ]:
import pandas, json, numpy, requests, os, datetime, statistics, math, pytz, tweepy, sqlite3, time, re, random, matplotlib.pyplot as plt, sklearn, statsmodels.api as sm
from bs4 import BeautifulSoup
from scipy.signal import find_peaks

This script merges parts of csv into a final table.

In [ ]:
def final_df_maker(symbol, name):
    try:
        ## Patterns_Score DataFrame will be the foundation DataFrame since it has the price data already included.
        patterns_df = pandas.read_csv(os.getcwd() + '\\Patterns Score\\'  + symbol + ' - ' + name + ' - Patterns Score.csv', index_col= 'Date', parse_dates= True)
        patterns_df = patterns_df.sort_index(ascending=True)
        merged_df = patterns_df.copy()

        print(symbol)

        base_5_df = pandas.read_csv(os.getcwd() + '\\Base Score\\'  + symbol + ' - ' + name + ' - Base Score - 5 Days.csv', index_col= 'Date', parse_dates= True)

        base_7_df = pandas.read_csv(os.getcwd() + '\\Base Score\\'  + symbol + ' - ' + name + ' - Base Score - 7 Days.csv', index_col= 'Date', parse_dates= True)

        base_10_df = pandas.read_csv(os.getcwd() + '\\Base Score\\'  + symbol + ' - ' + name + ' - Base Score - 10 Day.csv', index_col= 'Date', parse_dates= True)

        base_summary_df = pandas.read_csv(os.getcwd() + '\\Base Score\\'  + symbol + ' - ' + name + ' - Base Score - Summary.csv', index_col= 'Date', parse_dates= True)

        earnings_df = pandas.DataFrame()

        times_df = pandas.DataFrame()
        google_df = pandas.DataFrame()

        times_exist = 0
        google_exist = 0

        if os.path.exists(os.getcwd() + '\\Earnings\\' + symbol + ' - '+ name +' - Earnings - 2016-2020.csv') == True:
            earnings_df = pandas.read_csv(os.getcwd() + '\\Earnings\\' + symbol + ' - '+ name +' - Earnings - 2016-2020.csv', index_col= 'Date', parse_dates= True)

        if os.path.exists(os.getcwd() + '\\Historical Articles\\' + name +'.csv') == True:
            times_df = pandas.read_csv(os.getcwd() + '\\Historical Articles\\' + name +'.csv')

            if 'Text_Value' in times_df.columns:
                times_exist = 1

        if os.path.exists(os.getcwd() + '\\Google Search - News\\' + name +'.csv') == True:
            google_df = pandas.read_csv(os.getcwd() + '\\Google Search - News\\' + name +'.csv')

            if 'Text_Value' in google_df.columns:
                google_exist = 1

        date_extract = re.compile('\d{4}-\d{2}-\d{2}')

        if times_exist == 1 and google_exist == 1:
            for q in [times_df, google_df]:
                for p, r in q.iterrows():
                    q.pub_date[p] = date_extract.match(r.pub_date).group(0)

            times_summary_df = pandas.DataFrame({'Date': list(times_df.groupby(['pub_date']).count().index), 'Number_Of_Times_Articles': list(times_df.groupby(['pub_date']).count().headline), 'Times_Score': list(times_df.groupby(['pub_date']).sum().Text_Value)})
            times_summary_df.Date = pandas.to_datetime(times_summary_df.Date)

            google_summary_df = pandas.DataFrame({'Date': list(google_df.groupby(['pub_date']).count().index), 'Number_Of_Google_Articles': list(google_df.groupby(['pub_date']).count().Company), 'Google_Score': list(google_df.groupby(['pub_date']).sum().Text_Value)})
            google_summary_df.Date = pandas.to_datetime(google_summary_df.Date)

        elif times_exist == 1 and google_exist != 1:
            for p, r in times_df.iterrows():
                times_df.pub_date[p] = date_extract.match(r.pub_date).group(0)

            times_summary_df = pandas.DataFrame({'Date': list(times_df.groupby(['pub_date']).count().index), 'Number_Of_Times_Articles': list(times_df.groupby(['pub_date']).count().headline), 'Times_Score': list(times_df.groupby(['pub_date']).sum().Text_Value)})
            times_summary_df.Date = pandas.to_datetime(times_summary_df.Date)

        elif times_exist != 1 and google_exist == 1:
            for p, r in google_df.iterrows():
                google_df.pub_date[p] = date_extract.match(r.pub_date).group(0)

            google_summary_df = pandas.DataFrame({'Date': list(google_df.groupby(['pub_date']).count().index), 'Number_Of_Google_Articles': list(google_df.groupby(['pub_date']).count().Company), 'Google_Score': list(google_df.groupby(['pub_date']).sum().Text_Value)})
            google_summary_df.Date = pandas.to_datetime(google_summary_df.Date)


        ### Merging columns:
        temp_df = merged_df.merge(base_5_df.Base_Score, how='left', left_on=merged_df.index, right_on=base_5_df.index)
        temp_df = temp_df.merge(base_7_df.Base_Score, how='left', left_on='key_0', right_on=base_7_df.index)
        temp_df = temp_df.merge(base_10_df.Base_Score, how='left', left_on='key_0', right_on=base_10_df.index)
        temp_df = temp_df.merge(base_summary_df.Trend, how='left', left_on='key_0', right_on=base_summary_df.index)
        temp_df = temp_df.merge(base_summary_df.Average_Slope, how='left', left_on='key_0', right_on=base_summary_df.index)
        temp_df = temp_df.merge(base_summary_df.Predicted_Value_For_End_Date, how='left', left_on='key_0', right_on=base_summary_df.index)


        if os.path.exists(os.getcwd() + '\\Earnings\\' + symbol + ' - '+ name +' - Earnings - 2016-2020.csv') == False:
            temp_df['Earnings_Score'] = numpy.NaN
            temp_df['EPS_Estimate'] = numpy.NaN
            temp_df['EPS_Reported'] = numpy.NaN
            temp_df['Surprise_Percentage'] = numpy.NaN
            temp_df['Surprise_Overall_Average'] = numpy.NaN

        else:
            temp_df = temp_df.merge(earnings_df[['Earnings_Score', 'EPS_Estimate', 'EPS_Reported', 'Surprise_Percentage', 'Surprise_Overall_Average']], how='left', left_on='key_0', right_on=earnings_df.index)


        if times_exist != 1:
            temp_df['Number_Of_Times_Articles'] = numpy.NaN
            temp_df['NYTimes_Score'] = numpy.NaN

        else:
            temp_df = temp_df.merge(times_summary_df[['Number_Of_Times_Articles', 'Times_Score']], how='left', left_on='key_0', right_on=times_summary_df.Date)

        if google_exist != 1:
            temp_df['Number_Of_Google_Articles'] = numpy.NaN
            temp_df['Google_Score'] = numpy.NaN

        else:
            temp_df = temp_df.merge(google_summary_df[['Number_Of_Google_Articles', 'Google_Score']], how='left', left_on='key_0', right_on=google_summary_df.Date)


        temp_df = temp_df[temp_df.columns.drop(list(temp_df.filter(regex='Unnamed')))]

        temp_df.columns = ['Date', 'Symbol', 'Name', 'Open', 'High',
               'Low', 'Close', 'Volume', 'SMA_30', 'SMA_60', 'Patterns_Score',
               'Base_Score_5', 'Base_Score_7', 'Base_Score_10', 'Trend', 'Average_Slope_5_7_10',
               'Predicted_Value_For_End_Date', 'Earnings_Score', 'EPS_Estimate',
               'EPS_Reported', 'Surprise_Percentage', 'Surprise_Overall_Average',
               'Number_Of_Times_Articles', 'NYTimes_Score',
               'Number_Of_Google_Articles', 'Google_Score']

        merged_df = temp_df.copy()

        final_df = merged_df.copy()

        final_df = final_df.loc[:, ['Date', 'Symbol', 'Name', 'Open', 'High', 'Low', 'Close', 'Volume',
               'SMA_30', 'SMA_60', 'Patterns_Score', 'Base_Score_5', 'Base_Score_7',
               'Base_Score_10', 'Trend', 'Average_Slope_5_7_10',
               'Predicted_Value_For_End_Date', 'Earnings_Score', 
               'Number_Of_Times_Articles', 'NYTimes_Score',
               'Number_Of_Google_Articles', 'Google_Score']]

        final_df = final_df.fillna(0)

        if not os.path.exists(os.getcwd() + '\\Final DataFrame\\'):
            os.makedirs(os.getcwd() + '\\Final DataFrame\\')

        merged_df.to_csv(os.getcwd() + '\\Final DataFrame\\'  + symbol + ' - ' + name + ' - Summary.csv')
        final_df.to_csv(os.getcwd() + '\\Final DataFrame\\'  + symbol + ' - ' + name + ' - Final DataFrame.csv')
    
    except Exception:
        print('Error: ', symbol)
        pass
    


In [ ]:
## Looping:
stocks_and_names_with_indices = pandas.read_csv('merged_NYSE_AMEX_removed_intercept_pattern.csv')

for x, y in stocks_and_names_with_indices.iterrows():
    try:
        final_df_maker(symbol=y['Symbol'], name=y['Description'])  
        
    except Exception:
        continue
